# Import libs

In [36]:
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd

# Plots
# ==============================================================================
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modeling and Forecasting
# ==============================================================================
from catboost import CatBoostRegressor


from sklearn.metrics import mean_squared_error

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

from joblib import dump, load
from tqdm.auto import tqdm

# Warnings configuration
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [37]:
# функция возвращающая график ряда в зависимости от указанной фичи и скважены
def plot_series(dataset, well_number):
    dataset.loc[dataset['y'] == well_number].plot(subplots=True,
                                                     figsize=(20, 24),
                                                     title=str(f'Скважина №{well_number}'),
                                                    layout=(10,2),
                                                    grid=1)

# Этот ноутбук основывается на первой части, здесб мы обучим наши модели только на y без exog переменных

# Data distribution for features

Здесь создаем словарь {номер скважины: датафрейм}, выкидываем столбцы, если данных меньше 50%, заполняем оставшиеся линейной интерполяцией

In [38]:
data = pd.read_csv('data/train.csv', index_col=[0], parse_dates=[0])
data

,Номер скважины,Дебит нефти,Давление забойное,x,y,Объем жидкости,Объем нефти,Активная мощность (ТМ),Время работы (ТМ),Газовый фактор рабочий (ТМ),Давление буферное,Давление забойное от Hд,Давление забойное от Pпр,Давление линейное (ТМ),Давление на входе ЭЦН (ТМ),Дебит газа (ТМ),Дебит газа попутного,Дебит жидкости (ТМ),Коэффициент мощности (ТМ)
datetime,,,,,,,,,,,,,,,,,,,
1990-08-01,0,19.939,NaN,18670.86,5714.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-08-02,0,19.939,39.064454,18670.86,5714.86,NaN,NaN,50.851351,9.600833,NaN,NaN,33.913336,39.064454,1.887872,30.293676,3843.746667,24.25,24.250000,98.534314
1990-08-03,0,21.172,39.064487,18670.86,5714.86,NaN,NaN,52.353846,NaN,NaN,NaN,33.806090,39.064487,1.885714,30.261774,3900.955000,25.75,25.833333,99.139785
1990-08-04,0,22.529,38.965297,18670.86,5714.86,NaN,NaN,51.242424,9.600000,NaN,NaN,33.695717,38.965297,1.875851,30.212768,3874.505000,27.40,27.933333,98.744318
1990-08-05,0,22.529,38.766822,18670.86,5714.86,NaN,NaN,50.910256,9.563889,NaN,NaN,33.695717,38.766822,1.873163,30.109119,3853.696667,27.40,27.400000,98.419689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992-04-07,105,7.646,122.180672,14230.22,22456.04,NaN,NaN,252.298969,24.000000,NaN,NaN,122.180672,99.535735,11.548286,20.536082,NaN,451.36,451.358333,72.876289
1992-04-08,105,7.639,122.182381,14230.22,22456.04,15472.802,161.526,251.647423,24.000000,NaN,NaN,122.182381,99.202518,11.556784,20.459184,NaN,450.97,450.972727,72.855670
1992-04-09,105,7.657,122.183551,14230.22,22456.04,NaN,NaN,251.713542,24.000000,NaN,13.0,122.183551,99.119048,11.583542,20.104167,NaN,452.03,452.027273,72.302083


In [39]:
def fill_missed_features_with_linear(data):
    features = list(data.columns.drop('y'))
    #data.plot(subplots=True,
           #figsize=(20, 24),
           #title=str(f'Before'),
           #layout=(10,2),
           #grid=1)
    for feature in features:
        data[feature] = data[feature].interpolate(method='linear')
    #data.plot(subplots=True,
           #figsize=(20, 24),
           #title=str(f'After'),
           #layout=(10,2),
           #grid=1)
    return data

In [40]:
def get_each_well_dict(data):
    wells = list(data["Номер скважины"].unique())
    data = data.drop(columns=['x', 'y'])
    data = data.rename(columns={'Дебит нефти': 'y'})
    well_dict = {well: data[data['Номер скважины'] == well] for well in wells}
        
    return well_dict

In [41]:
wells = get_each_well_dict(data)

Clean duplicated indexes, columns with less than 50% of data

In [42]:
for i in range(len(wells)):
    wells[i] = wells[i][~wells[i].index.duplicated(keep='first')]
    wells[i] = wells[i].asfreq(freq ='D')
    wells[i] = wells[i].dropna(axis=1, thresh=int(0.5*len(wells[i])))
    wells[i] = impute(wells[i])

# Test on first well

Зададим даты тренировочного и тестового протестируемся на 1ой скважине

In [43]:
end_train = '1992-04-10'
start_test = '1992-04-11'

In [49]:
best_params = {}

In [50]:
import json

with open("best_param_grid_new.json", "r") as file:
    best_params = json.load(file) 

## Predict for each well

In [51]:
best_params[str(1)]

{'n_lags': 80,
 'params': {'learning_rate': 0.12, 'max_depth': 7, 'n_estimators': 600}}

In [52]:
def get_forecast(wells_dict, params_dict):
    all_predictions = pd.DataFrame()
    for i in tqdm(range(len(wells))):
        print(f'Predition well №{i}')
        
        data = wells[i].copy()
        data_train = data.loc[:end_train, :]
        
        
        forecaster = ForecasterAutoreg(
            regressor = CatBoostRegressor(**best_params[str(i)]['params'], silent=True),
            lags = best_params[str(i)]['n_lags'])
        
        forecaster.fit(y=data_train['y'])
        
        steps=90
        
        predictions = pd.DataFrame()
        predictions['forecast'] = forecaster.predict(steps=steps)
        predictions['Номер скважины'] = i
        predictions['datetime'] = predictions.index

        predictions = predictions[['datetime', 'forecast', 'Номер скважины']]
        
        all_predictions = pd.concat([all_predictions, predictions], ignore_index=False, sort=False)
        
    return all_predictions

In [53]:
well_predictions = get_forecast(wells, best_params)

  0%|          | 0/106 [00:00<?, ?it/s]

Predition well №0
Predition well №1
Predition well №2
Predition well №3
Predition well №4
Predition well №5
Predition well №6
Predition well №7
Predition well №8
Predition well №9
Predition well №10
Predition well №11
Predition well №12
Predition well №13
Predition well №14
Predition well №15
Predition well №16
Predition well №17
Predition well №18
Predition well №19
Predition well №20
Predition well №21
Predition well №22
Predition well №23
Predition well №24
Predition well №25
Predition well №26
Predition well №27
Predition well №28
Predition well №29
Predition well №30
Predition well №31
Predition well №32
Predition well №33
Predition well №34
Predition well №35
Predition well №36
Predition well №37
Predition well №38
Predition well №39
Predition well №40
Predition well №41
Predition well №42
Predition well №43
Predition well №44
Predition well №45
Predition well №46
Predition well №47
Predition well №48
Predition well №49
Predition well №50
Predition well №51
Predition well №52
Pre

In [54]:
well_predictions.to_csv('CatBoost_forecast_without_exog.csv', index=False, encoding='utf=8')